In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split

In [2]:
data = pd.read_pickle('standardized_base.pkl')
to_scam = pd.read_pickle('to_scam.pkl')
from_scam = pd.read_pickle('from_scam.pkl')

In [3]:
# get single column indicating if transactions is fraudulent
temp = to_scam + from_scam
scam =[]
for i in temp:
    if i==0:
        scam.append(0)
    else:
        scam.append(1)
target = pd.DataFrame(scam)
target

,0
0,0
1,0
2,0
3,0
4,0
...,...
71245,1
71246,1
71247,1
71248,1


# Deep Neural Networks

In [4]:
# import DNN libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras import Sequential 

In [5]:
# create DNN maker function
def create_dnn(nodes,layers,dropout,activation):
    model = Sequential()
    
    for i in list(range(1,layers+1)):
        if i == 1:
            model.add(Dense(nodes, input_dim = (data.shape[1]), activation = 'swish'))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))
        else:
            model.add(Dense(i/2, activation='swish'))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))

    model.add(BatchNormalization())
    model.add(Dense(1))
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer = 'Adam')
    return model

In [ ]:
models_base_dnn = {}

kfold = KFold(n_splits=5, shuffle=True)
kfold.get_n_splits(data)

splt=0
for train_index, test_index in kfold.split(data):
    x_train, x_test = np.array(data)[train_index], np.array(data)[test_index]
    y_train, y_test = np.array(target)[train_index], np.array(target)[test_index]

    for i in [64,32,16]:
        for j in [43,2,1]:
                model = create_dnn(i,j,0.3,'swish') # create a DNN with the given number of initial nodes and layers
            
                model.fit(x_train, pd.DataFrame(y_train), epochs = 300, batch_size = 4096, validation_data=(x_test,pd.DataFrame(y_test)),verbose = 1) # fit the DNN
                
                comb = str(i)+'-'+str(j)+'-'+str(splt)
                models[comb] = model # store the model
    splt+=1

Epoch 1/300
14/14 [==============================] - 0s 13ms/step - loss: 0.7069 - val_loss: 0.6893
Epoch 2/300
14/14 [==============================] - 0s 5ms/step - loss: 0.6894 - val_loss: 0.6859
Epoch 3/300
14/14 [==============================] - 0s 6ms/step - loss: 0.6618 - val_loss: 0.6806
Epoch 4/300
14/14 [==============================] - 0s 6ms/step - loss: 0.6414 - val_loss: 0.6740
Epoch 5/300
14/14 [==============================] - 0s 6ms/step - loss: 0.6287 - val_loss: 0.6708
Epoch 6/300
14/14 [==============================] - 0s 6ms/step - loss: 0.6170 - val_loss: 0.6757
Epoch 7/300
14/14 [==============================] - 0s 6ms/step - loss: 0.6050 - val_loss: 0.6889
Epoch 8/300
14/14 [==============================] - 0s 6ms/step - loss: 0.5933 - val_loss: 0.7069
Epoch 9/300
14/14 [==============================] - 0s 6ms/step - loss: 0.5840 - val_loss: 0.7220
Epoch 10/300
14/14 [==============================] - 0s 6ms/step - loss: 0.5746 - val_loss: 0.7286
Epoch 11

In [8]:
for i in models_base_dnn:
        models_base_dnn[i]

{'64-3-0': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fb76c1e3990>,
 '64-2-0': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fb76c077490>,
 '64-1-0': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fb76b417410>,
 '32-3-0': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fb76b023bd0>,
 '32-2-0': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fb76ab9c850>,
 '32-1-0': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fb76a804d90>,
 '16-3-0': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fb76b05cfd0>,
 '16-2-0': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fb787509890>,
 '16-1-0': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fb76b5848d0>,
 '64-3-1': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fb781756cd0>,
 '64-2-1': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fb76f5ba990>,
 '64-1-1': <tensorflow.python.keras.engine.sequential.Sequential 

In [16]:
mod = models_base_dnn['16-1-4']

In [23]:
mod.